In [95]:
using InvertibleNetworks, Flux
import Flux.Optimise.update!
using Flux: onecold
using CSV
using DataFrames
using Lathe.preprocess: TrainTestSplit

# Define network
n_in = 2
n_hidden = 64
depth = 4
AN = Array{ActNorm}(undef, depth)
modelLayers = Array{CouplingLayerGlow}(undef, depth)
Params = Array{Parameter}(undef, 0)

# Create layers
for j=1:depth
    AN[j] = ActNorm(n_in; logdet=true)
    modelLayers[j] = CouplingLayerGlow(n_in, n_hidden; k1=1, k2=1, p1=0, p2=0, logdet=true)

    # Collect parameters
    global Params = cat(Params, get_params(AN[j]); dims=1)
    global Params = cat(Params, get_params(modelLayers[j]); dims=1)
end


df = CSV.read("dataset.csv",delim=';',DataFrame,missingstring="?",
             silencewarnings=true,ntasks=1)

train, test = TrainTestSplit(df,.75)
println("Train data size: ",size(train))
println("Test data size: ",size(test))
#CSV.write("train_data.csv",train)
#CSV.write("test_data.csv",test)

function addZeros(A,B)
    countZeros = 0
    if length(A) > length(B)
        dif = length(A) - length(B)
        while countZeros < dif
            push!(B,0)
            countZeros = countZeros + 1 
        end    
    elseif length(A) < length(B)
        dif2 = length(B) - length(A)
        while countZeros < dif2
            push!(A,0)
            countZeros = countZeros + 1
        end
    end
end


# Forward pass of invertible structure
function forward(X)
    logdet = 0f0
    for j=1:depth
        X_, logdet1 = AN[j].forward(X)
        X, logdet2 = modelLayers[j].forward(X_)
        logdet += (logdet1 + logdet2)
    end
    return X, logdet
end

# Backward pass of invertible structure
function backward(ΔX, X)
    for j=depth:-1:1
        ΔX_, X_ = modelLayers[j].backward(ΔX, X)
        ΔX, X = AN[j].backward(ΔX_, X_)
    end
    return ΔX, X
end

####################################################################################################

# Loss calculation
function loss(X)
    Y, logdet = forward(X)
    f = -log_likelihood(Y) - logdet
    ΔY = -∇log_likelihood(Y)
    ΔX = backward(ΔY, Y)[1]
    return f, ΔX
end

function accuracy(x, y)
    return sum(onecold(x) .== onecold(y))
end
# Training with 10517 data
df_train = CSV.read("train_data.csv",delim=';',DataFrame,missingstring="?",
             silencewarnings=true,ntasks=1)

maxiter = size(df_train)[1]
opt = Flux.ADAM(1f-3)
fval = zeros(Float32, maxiter)

for j=1:(maxiter-1)
    T = df_train[!,"Set1"][j]
    T2 = df_train[!,"Set1"][j+1]
    C = df_train[!,"Set2"][j]
    C2 = df_train[!,"Set2"][j+1]
    VectorT = []
    VectorC = []
    VectorT2 = []
    VectorC2 = []
    T = chop(T, head = 1, tail = 1)
    C = chop(C, head = 1, tail = 1)
    T2 = chop(T2, head = 1, tail = 1)
    C2 = chop(C2, head = 1, tail = 1)
    T = rsplit(T,",")
    C = rsplit(C,",")
    T2 = rsplit(T2,",")
    C2 = rsplit(C2,",")
    for i in T
        i = parse(Int64,i)
        push!(VectorT,i)
    end
    for t in C
        t = parse(Int64,t)
        push!(VectorC,t)
    end
    for i in T2
        i = parse(Int64,i)
        push!(VectorT2,i)
    end
    for t in C2
        t = parse(Int64,t)
        push!(VectorC2,t)
    end
        
    output_length = length(VectorC)
    
    addZeros(VectorT,VectorC)
    addZeros(VectorT2,VectorC2)
    addZeros(VectorT,VectorC2)
    addZeros(VectorC,VectorC2)
    addZeros(VectorT,VectorT2)
    addZeros(VectorC,VectorC2)
    #default batch size = 2
    X = [VectorT;;;VectorT2;;;;VectorC;;;VectorC2;;;]  
    fval[j] = loss(X)[1]
    mod(j, 20) == 0 && (print("Iteration: ", j, "; f = ", fval[j], "\n"))

    # Update params
    for p in Params
        update!(opt, p.data, p.grad)
    end
    clear_grad!(Params)

end

####################################################################################################



Train data size: (10485, 2)
Test data size: (3451, 2)
Iteration: 20; f = 35.528564
Iteration: 40; f = 56.015266
Iteration: 60; f = 24.66853
Iteration: 80; f = 30.051605
Iteration: 100; f = 36.53155
Iteration: 120; f = 50.03912
Iteration: 140; f = 18.344173
Iteration: 160; f = 33.10005
Iteration: 180; f = 21.946505
Iteration: 200; f = 46.976837
Iteration: 220; f = 40.636787
Iteration: 240; f = 32.43722
Iteration: 260; f = 23.508415
Iteration: 280; f = 24.429142
Iteration: 300; f = 23.60307
Iteration: 320; f = 31.66708
Iteration: 340; f = 19.811388
Iteration: 360; f = 20.451668
Iteration: 380; f = 27.833126
Iteration: 400; f = 26.480284
Iteration: 420; f = 23.416529
Iteration: 440; f = 20.645899
Iteration: 460; f = 32.045654
Iteration: 480; f = 25.379353
Iteration: 500; f = 37.220093
Iteration: 520; f = 26.32993
Iteration: 540; f = 37.04762
Iteration: 560; f = 29.007034
Iteration: 580; f = 25.063976
Iteration: 600; f = 32.936756
Iteration: 620; f = 25.557709
Iteration: 640; f = 25.790747

Iteration: 6680; f = 27.750399
Iteration: 6700; f = 45.48472
Iteration: 6720; f = 39.62579
Iteration: 6740; f = 39.964115
Iteration: 6760; f = 38.250114
Iteration: 6780; f = 33.725155
Iteration: 6800; f = 29.48166
Iteration: 6820; f = 39.357685
Iteration: 6840; f = 29.964836
Iteration: 6860; f = 38.512856
Iteration: 6880; f = 34.938652
Iteration: 6900; f = 49.772274
Iteration: 6920; f = 25.47156
Iteration: 6940; f = 22.85686
Iteration: 6960; f = 42.144596
Iteration: 6980; f = 35.058678
Iteration: 7000; f = 24.531923
Iteration: 7020; f = 21.782616
Iteration: 7040; f = 32.337284
Iteration: 7060; f = 40.14687
Iteration: 7080; f = 37.09492
Iteration: 7100; f = 32.8001
Iteration: 7120; f = 41.663403
Iteration: 7140; f = 39.782448
Iteration: 7160; f = 43.9154
Iteration: 7180; f = 30.203133
Iteration: 7200; f = 17.630527
Iteration: 7220; f = 63.193615
Iteration: 7240; f = 54.876366
Iteration: 7260; f = 41.884144
Iteration: 7280; f = 34.40835
Iteration: 7300; f = 25.324768
Iteration: 7320; f =

In [96]:


# Testing with 3495 data
df_test = CSV.read("test_data.csv",delim=';',DataFrame,missingstring="?",
             silencewarnings=true,ntasks=1)

acc = 0
count = 0
loop_control = length(df_test[!,"Set1"]) 
Dict1 = Dict()
keepOutput = []
keepInput = []
keepInvertibleResult = []
for ind=1:(loop_control-1)    
    A = df_test[!,"Set1"][ind]
    push!(keepInput,A)
    B = df_test[!,"Set2"][ind]
    push!(keepOutput,B)
    VectorA = []
    VectorB = []
    VectorC = []
    A = chop(A, head = 1, tail = 1)
    B = chop(B, head = 1, tail = 1)
    A = rsplit(A,",")
    B = rsplit(B,",")
    for i in A
        i = parse(Int64,i)
        push!(VectorA,i)
    end
    for t in B
        t = parse(Int64,t)
        push!(VectorB,t)
        push!(VectorC,t)
    end
    output_length = length(VectorB)
    addZeros(VectorA,VectorB)
    
    X = [VectorA;;;VectorB;;;;]
    
    VectorC = VectorB
    #dropzeros!(VectorC)
    VectorC = filter(!iszero, VectorC)
    Y_ = forward(X)[1]
    X_ = backward(Y_,Y_)[2]
    input_length = length(VectorA)
    push!(Dict1, VectorA => X_[input_length+1:input_length + output_length])
    push!(keepInvertibleResult,X_[input_length+1:input_length + output_length])
    push!(Dict1,VectorC => X_[1:input_length])
    count = count + 1
    acc += accuracy(X[input_length+1:input_length + output_length],X_[input_length+1:input_length + output_length])
    if ind == loop_control-1
        println("accuracy: %",(acc/count)*100)
    end
end  

function learningBijections(Vec)
    return println(Dict1[Vec])
end

        



accuracy: %89.20717592592592


learningBijections (generic function with 1 method)

In [97]:
# INVERTIBILITY TEST BETWEEN SET1 AND SET2
# Set1: [1,3,3,3]
# Set2: [3,9]
# Test for input->output
learningBijections([1,3,3,3])
#Test for output-> input
learningBijections([1,9])

println("****************************************")
# Set1: [1,3,15]
# Set2: [1,5,13] 
# Test for input->output
learningBijections([1,3,15])  # [1,3,15] -> [1,5,13] 
#Test for output-> input
learningBijections([1,5,13]) # [1,5,13] -> [1,3,15]  

[1.000000000000009, 8.999999999999995]
[1.0000000000000002, 2.9999999999999964, 3.0000000000000004, 3.0000000000000004]
****************************************
[1.0000000000000093, 5.000000000000003, 13.000000000000007]
[0.9999999999999997, 2.9999999999999964, 14.999999999999984]
